<a href="https://colab.research.google.com/github/ihshin227/tensorlite_convert_test/blob/master/Keras_CNN_convert_to_tensorflow_lite_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Model configuration
batch_size = 150
img_width, img_height = 28, 28
loss_function = sparse_categorical_crossentropy
no_classes = 10
no_epochs = 25
optimizer = Adam()
validation_split = 0.2
verbosity = 1

# Load MNIST dataset
(input_train, target_train), (input_test, target_test) = mnist.load_data()

# Reshape data    
input_train = input_train.reshape((input_train.shape[0], img_width, img_height, 1))
input_test = input_test.reshape((input_test.shape[0], img_width, img_height, 1))
input_shape = (img_width, img_height, 1)

# Cast input to float32
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

# Compile the model
model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])

# Train the model
model.fit(input_train, target_train,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split=validation_split)


Epoch 1/25
320/320 [==============================] - 5s 16ms/step - loss: 0.1362 - accuracy: 0.9587 - val_loss: 0.0562 - val_accuracy: 0.9838
Epoch 2/25
320/320 [==============================] - 5s 16ms/step - loss: 0.0362 - accuracy: 0.9885 - val_loss: 0.0427 - val_accuracy: 0.9874
Epoch 3/25
320/320 [==============================] - 5s 16ms/step - loss: 0.0188 - accuracy: 0.9934 - val_loss: 0.0558 - val_accuracy: 0.9860
Epoch 4/25
320/320 [==============================] - 5s 16ms/step - loss: 0.0131 - accuracy: 0.9958 - val_loss: 0.0474 - val_accuracy: 0.9882
Epoch 5/25
320/320 [==============================] - 5s 16ms/step - loss: 0.0080 - accuracy: 0.9974 - val_loss: 0.0659 - val_accuracy: 0.9840
Epoch 6/25
320/320 [==============================] - 5s 16ms/step - loss: 0.0080 - accuracy: 0.9972 - val_loss: 0.0547 - val_accuracy: 0.9887
Epoch 7/25
320/320 [==============================] - 5s 16ms/step - loss: 0.0061 - accuracy: 0.9981 - val_loss: 0.0477 - val_accuracy: 0.9903

In [7]:
from tensorflow.keras.models import Sequential, save_model

# Save the model
filepath = './saved_model'
save_model(model, filepath)


INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


In [0]:
converter = tf.lite.TFLiteConverter.from_saved_model(filepath)

In [0]:
tflite_model = converter.convert()

In [0]:
import numpy as np

interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

tflite_results = interpreter.get_tensor(output_details[0]['index'])
tf_results = model(tf.constant(input_data))